In [1]:
import pandas as pd
import dask.dataframe as dd
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Extract Features

In [ ]:
df = pd.read_pickle('df_feature.pkl')

extracted_features = extract_features(df, column_id='id', column_sort='time', default_fc_parameters= ComprehensiveFCParameters(), impute_function=impute)

extracted_features.to_pickle('df_feature_all.pkl')


# Feature Selection

In [8]:
import pandas as pd
import dask.dataframe as dd
from tsfresh import extract_features, select_features

ImportError: cannot import name 'calculate_relevance_table' from 'tsfresh.feature_selection' (/Users/felixschreyer/anaconda3/envs/Testing/lib/python3.11/site-packages/tsfresh/feature_selection/__init__.py)

In [3]:
df = pd.read_pickle('df_feature.pkl')
X = pd.read_pickle('df_feature_all.pkl')

In [4]:
y_s_train_distinct = df.groupby('id')['RUL_rolled_class'].last()


In [10]:
from tsfresh.feature_selection.relevance import calculate_relevance_table

table = calculate_relevance_table(X, y_s_train_distinct, ml_task='classification', multiclass=True, show_warnings=False)

In [37]:
table

,feature,type,p_value_1,relevant_1,p_value_2,relevant_2,p_value_3,relevant_3,n_significant,relevant
feature,,,,,,,,,,
x__number_cwt_peaks__n_5,x__number_cwt_peaks__n_5,real,1.181918e-08,True,0.000002,True,0.003197,False,2,True
y__number_peaks__n_10,y__number_peaks__n_10,real,1.235788e-08,True,0.000002,True,0.003196,False,2,True
y__number_cwt_peaks__n_1,y__number_cwt_peaks__n_1,real,1.236191e-08,True,0.000002,True,0.003196,False,2,True
x__number_peaks__n_5,x__number_peaks__n_5,real,1.236593e-08,True,0.000002,True,0.003197,False,2,True
x__range_count__max_1__min_-1,x__range_count__max_1__min_-1,real,1.264865e-08,True,0.000002,True,0.003176,False,2,True
...,...,...,...,...,...,...,...,...,...,...
y__number_crossing_m__m_-1,y__number_crossing_m__m_-1,constant,NaN,False,NaN,False,NaN,False,0,False
y__number_crossing_m__m_1,y__number_crossing_m__m_1,constant,NaN,False,NaN,False,NaN,False,0,False
y__count_above__t_0,y__count_above__t_0,constant,NaN,False,NaN,False,NaN,False,0,False


In [34]:
lowest_indices_column1 = table['p_value_1'].nsmallest(15).index
lowest_indices_column2 = table['p_value_2'].nsmallest(15).index
lowest_indices_column3 = table['p_value_3'].nsmallest(15).index

# Combine the indices to get the unique rows with the lowest values in any column
lowest_indices_combined = lowest_indices_column1.union(lowest_indices_column2).union(lowest_indices_column3)

# Filter the original DataFrame to keep only the rows with the lowest values
filtered_df = table.loc[lowest_indices_combined]
filtered_df
#lowest_indices_combined

,feature,type,p_value_1,relevant_1,p_value_2,relevant_2,p_value_3,relevant_3,n_significant,relevant
feature,,,,,,,,,,
x__abs_energy,x__abs_energy,real,2.771913e-08,True,0.000004,True,4.987593e-06,True,3,True
x__cid_ce__normalize_True,x__cid_ce__normalize_True,real,1.417667e-08,True,0.000002,True,4.987593e-06,True,3,True
x__count_below_mean,x__count_below_mean,real,1.354402e-08,True,0.000002,True,3.196998e-03,False,2,True
"x__fft_coefficient__attr_""abs""__coeff_0","x__fft_coefficient__attr_""abs""__coeff_0",real,1.483081e-08,True,0.000002,True,4.987593e-06,True,3,True
"x__fft_coefficient__attr_""real""__coeff_0","x__fft_coefficient__attr_""real""__coeff_0",real,1.483081e-08,True,0.000002,True,4.987593e-06,True,3,True
x__length,x__length,real,1.264865e-08,True,0.000002,True,3.175847e-03,False,2,True
x__number_cwt_peaks__n_1,x__number_cwt_peaks__n_1,real,1.354347e-08,True,0.000002,True,3.196961e-03,False,2,True
x__number_cwt_peaks__n_5,x__number_cwt_peaks__n_5,real,1.181918e-08,True,0.000002,True,3.196663e-03,False,2,True
x__number_peaks__n_3,x__number_peaks__n_3,real,1.353635e-08,True,0.000002,True,3.196477e-03,False,2,True


In [39]:
lowest_indices_combined

Index(['x__abs_energy', 'x__cid_ce__normalize_True', 'x__count_below_mean',
       'x__fft_coefficient__attr_"abs"__coeff_0',
       'x__fft_coefficient__attr_"real"__coeff_0', 'x__length',
       'x__number_cwt_peaks__n_1', 'x__number_cwt_peaks__n_5',
       'x__number_peaks__n_3', 'x__number_peaks__n_5',
       'x__range_count__max_1000000000000.0__min_0',
       'x__range_count__max_1__min_-1', 'x__ratio_beyond_r_sigma__r_10',
       'x__sum_of_reoccurring_data_points', 'x__sum_of_reoccurring_values',
       'x__sum_values', 'y__cid_ce__normalize_True',
       'y__fft_coefficient__attr_"abs"__coeff_0',
       'y__fft_coefficient__attr_"real"__coeff_0', 'y__length',
       'y__number_cwt_peaks__n_1', 'y__number_peaks__n_1',
       'y__number_peaks__n_10', 'y__number_peaks__n_3',
       'y__percentage_of_reoccurring_datapoints_to_all_datapoints',
       'y__percentage_of_reoccurring_values_to_all_values',
       'y__permutation_entropy__dimension_6__tau_1',
       'y__range_count__max

In [19]:
X_selected = select_features(X, y_s_train_distinct, ml_task='classification', multiclass=True, show_warnings=True, fdr_level= 0.00001)

/Users/felixschreyer/anaconda3/envs/Testing/lib/python3.11/site-packages/tsfresh/feature_selection/relevance.py:222: RuntimeWarning: [test_feature_significance] Constant features: x__variance_larger_than_standard_deviation, x__has_duplicate_max, x__has_duplicate_min, x__has_duplicate, x__symmetry_looking__r_0.0, x__symmetry_looking__r_0.05, x__symmetry_looking__r_0.1, x__symmetry_looking__r_0.15000000000000002, x__symmetry_looking__r_0.2, x__symmetry_looking__r_0.25, x__symmetry_looking__r_0.30000000000000004, x__symmetry_looking__r_0.35000000000000003, x__symmetry_looking__r_0.4, x__symmetry_looking__r_0.45, x__symmetry_looking__r_0.5, x__symmetry_looking__r_0.55, x__symmetry_looking__r_0.6000000000000001, x__symmetry_looking__r_0.65, x__symmetry_looking__r_0.7000000000000001, x__symmetry_looking__r_0.75, x__symmetry_looking__r_0.8, x__symmetry_looking__r_0.8500000000000001, x__symmetry_looking__r_0.9, x__symmetry_looking__r_0.9500000000000001, x__large_standard_deviation__r_0.2, x__l

In [45]:
X_selected = X.loc[:, X.columns.isin(lowest_indices_combined)]

In [46]:
X_selected.to_pickle('df_feature_selected.pkl')

# Relevant Features S Train

In [2]:
import tsfresh
import pandas as pd

In [3]:
df_s_train = pd.read_pickle('df_t_train_rolled.pkl')
kind_to_fc_parameters = tsfresh.feature_extraction.settings.from_columns(pd.read_pickle('df_feature_selected.pkl'))

In [4]:
df_s_train = df_s_train.drop(columns=['time_end', 'bearing_id', 'RUL_rolled_class', 'RUL_rolled'])
#ddf_s_train = dd.from_pandas(df_s_train, npartitions=20)

In [ ]:
del [ddf_helper]
del [df_s_train]

In [ ]:
X_s_train   = extract_features(df_s_train, column_id='id', column_sort='time', kind_to_fc_parameters= kind_to_fc_parameters)

Feature Extraction:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
X_s_train.to_pickle('X_t_train.pkl')